# Benchmarking by comparing to analytical and theoretical results

## TODO :

Analyse des premiers résultats :
- variation assez élevée du nombre de particules quittant le système à un instant donné (<25, mais varie entre 5 et 25)
- les débits massiques en sortie ne correspondent pas (mais ça peut être normal, pour le coup ça dépend pas mal de données rentrées sur le site)
Pour la suite, je vais essayer de comparer sur plusieurs nombre de particules par cellule : 100 - 200 - 500 - 1000 - 2000 - 5000 - 10000, pour commencer, sur environ $10^{-4}$ secondes. Soit, avec $dt = 5 \times 10^{-7}$, on doit faire 10000 pas de temps.

Avant cela, il y a également le problème des pas de temps et d'espace. Dans cet [article](https://www.sciencedirect.com/science/article/pii/S0045793015002558), on voit que les contraintes sont au nombre de quatre et plus subtiles que ce que j'ai considéré jusqu'à présent.


## Comparison with flow in tube

[Link](http://fisica.ufpr.br/sharipov/tube.html)


Speed of sound : 
 $$ v = \sqrt{\frac{\gamma R T_K}{M}}$$
with : 
* $\gamma$ : the adiabatic index
* $R = 8.314 \text{ J.K.mol}^{-1}$ the gas constant
* $T_K$ the temperature in Kelvin
* $M$ the molecular mass


For our case with Iodine $[I]$ :
* $\gamma = \frac{5}{3}$
* $T_K = 300 \text{ K}$
* $M \approx 126.9 \text{ g.mol}^{-1}$

Which yields : $v \approx  \text{ m.s}^{-1}$

In [1]:
import numpy as np
def speed_of_sound(molecular_mass, temperature, gamma):
    # all units in SI
    return np.sqrt(8.314*gamma*temperature/molecular_mass)

In [2]:
gamma = 5/3.
molecular_mass = 0.1269 # kg/mol
temperature = 300 # K
v = speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for iodine [I] at temperature 300 K : {v} m/s')

Speed of sound for iodine [I] at temperature 300 K : 180.99192585903475 m/s


### Definition of the tube

Imports : 

In [21]:
%matplotlib widget

# system
from src.system_creator import SystemCreator

# Grid
from src.utils import Grid, pos_in_grid, convert_to_grid_datatype

# Particles
from src.utils import Particle

# injection 
from src.utils import inject

# advection
from src.utils import advect
from src.utils import euler_explicit, leap_frog

# collisions
from src.utils import handler_wall_collision, handler_wall_collision_point, make_collisions_vectorized, make_collisions_out_walls, deal_with_corner

# utils 
from src.utils import gaussian, maxwellian_flux, maxwellian_mean_speed, get_mass_part, mean_free_path, mean_free_time

# systems
from src.utils import systems

# plotting 
from src.plotting import plot_boundaries, plot_particles, plot_grid, plot_system
from src.plotting import analysis

# collisions between particles
from src.utils import handler_particles_collisions, candidates # candidates, index_choosen_couples, probability, is_colliding, reflect, 

# saving 
from src.data import Saver

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import os
np.random.seed(1111)

In [22]:
# Square System :
dz = 0.001
idx_out_walls = [1,2] # no out walls

segments = 0.001*3*np.array([[0,0,10,0], [0,0,0,1], [10,0,10,1], [0,1,10,1]]) # bottom, left, right, top
system = SystemCreator(segments)

offsets = system.get_offsets()
system_shape = system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()
extremal_values = system.get_extremal_values()

fig, ax = plt.subplots()
ax.axis('equal')
plot_boundaries(ax, segments)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
AVOGADRO_CONSTANT = 6.02e23 # mol-1

# grid :
mean_number_per_cell = 100
max_number_per_cell = 10*mean_number_per_cell
resolutions = np.array((20,2), dtype = int) # tube
grid = Grid(resolutions, max_number_per_cell)
volume_cell = dz * system_shape[0]/resolutions[0] * system_shape[1]/resolutions[1]

# Particles - 1 type 
density = 3.2e19 # m-3
n_simu = mean_number_per_cell*np.prod(resolutions) # number of particles in the simulated system
n_real = volume_cell * density * np.prod(resolutions) # number of particles in the real system
mr = n_real/n_simu # macro particules ratio = number of particles in the real system / number of macro part in the simulated system
density_dsmc = density/mr
temperature = 300 # K

part_type = 'I'
charge, mass, radius = 0, get_mass_part(53, 53, 74), 2e-10
size_array = 2*mean_number_per_cell*np.prod(resolutions)
v_mean = maxwellian_mean_speed(temperature, mass)
container = Particle(part_type, charge, mass, radius, size_array)
cross_section = container.get_params()[3]

    # mean free path and time
mfp = mean_free_path(cross_section, density)
typical_lenght = 0.001
mft = mean_free_time(typical_lenght, v_mean = v_mean)

    # Injection params
in_wall = np.array([0,0,0,0.001], dtype = float) # tube
in_vect = np.array([1,0], dtype = float)

debit = maxwellian_flux(density_dsmc, v_mean)*np.linalg.norm(in_wall[:2]-in_wall[2:])*dz # particles per second
vel_std = gaussian(temperature, mass)

    # Simulation params
iterations = 1000
dt = 1e-5 # in sec, should be a fraction of the mean free time
    
    # saving params
saving_period = 100
adding_period = 10

    # advection
def f(arr, dt):
    return np.zeros(shape = (arr.shape[0], 3))
args = []
scheme = euler_explicit

In [6]:
print('Input mass flow rate : {:.3e} kg/s'.format(mr*debit*mass))
print('Equivalent pressure : {:.3e} Pa'.format(density*8.314*temperature/AVOGADRO_CONSTANT))
print('Fraction between the mean deplacement per time step and the mean free path : {:e} '.format(dt*v_mean/mfp))

Input mass flow rate : 3.786e-10 kg/s
Equivalent pressure : 1.326e-01 Pa
Fraction between the mean deplacement per time step and the mean free path : 3.584441e-02 


In [7]:
# NAME tests
from pathlib import Path

dir_path = Path('results/benchmark/')
name = 'tube-flow.h5'

saver = Saver(dir_path, name)

In [8]:
df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) # bucket for the particles - index of particles is the iteration number

# defining useful arrays and ints 
remains = 0 # fractionnal part of the number of particles to inject (it is then passed to the following time step)
averages = np.full(shape = grid.current.shape, fill_value = mean_number_per_cell) # average number of particles per cell
pmax = 2*v_mean*cross_section*np.ones(averages.shape) # max proba per cell in the simu
remains_per_cell = np.zeros(shape = grid.current.shape, dtype = float) # remains per cell for the particles collisions step

# SIMULATING
print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', ' INJECT ', ' DEL ', ' TRY'))
print('{:-^56}'.format(''))

for it in range(iterations): # tqdm
    n1 = container.get_current()
                   
    # injecting particles
    new, remains = inject(in_wall, in_vect, debit, vel_std, radius, dt, remains)
    container.add_multiple(new)
                   
    n2 = container.get_current()-n1
    
    # PHASE : ADVECTING
        # MOVING PARTICLES
    arr = container.get_particles()
    
    if(it%adding_period == 0):
        df = df.append(pd.DataFrame(data=arr, index=[it]*arr.shape[0], columns = ['x','y','vx','vy','vz']))
    
    advect(arr, f, dt, args, scheme) # advect is inplace
    
        # HANDLING BOUNDARIES 
    count = np.full(fill_value = True, shape = arr.shape[0])
    idxes_out = []
    c = 0
    while(np.sum(count, where = count == True) > 0):
        c+=1
        ct, cp = handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_ = make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)
    
    idxes_out = np.concatenate(idxes_out)
    
    # TODO : make delete multiple better - currently the function creates a new array where as we can do it inplace.
    container.delete_multiple(idxes_out)
    
    arr = container.get_particles()
    
    # PHASE : COLLISIONS
        # UPDATING GRID - HARD RESET
        # TODO : change the way it's done
    grid.reset()
    positions = pos_in_grid(arr[:,:2], resolutions, offsets, system_shape)
    particles = convert_to_grid_datatype(positions, new = positions.shape[0])
    grid.add_multiple(particles)
        
        # DSMC
        # TODO: make parallel
    currents = grid.get_currents()
    averages = (it*averages+currents)/(it+1) # may be it too violent ? 
    
    remains_per_cell, nb_colls, pmax, monitor = handler_particles_collisions([arr], grid.get_grid(), currents, dt, averages, pmax, cross_section, volume_cell, mr, remains_per_cell, monitoring = True)
    # PLOTTING AND SAVING (OPTIONAL)
    if(it%saving_period==0 or it == iterations-1): # saving if last iterations too
        saver.save(it = it, append = {
                        'df' : df,
                        'collisions_per_cell' : nb_colls, # evolution of the number of collisions per cell - size : grid.shape[0] x grid.shape[1] (2D)
                        'total_distance' : float(monitor[0]), # evolution of the sum of the distance accross all cells 
                        'total_proba' : float(monitor[1]), # evolution of the sum of proba accross all cells
                        'pmax_per_cell' : pmax,  # evolution of the sum of pmax - per cell (2D)
                        'total_deleted' : len(idxes_out), # evolution of the number of deleted particles per cell (int)
                        'averages_per_cell' : averages
                  })
        
        # resetting dataframe to not use too much memory
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) 
        print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(it, n1, n2, idxes_out.shape[0], c))
   
saver.close()

|    it    |   INIT   |  INJECT  |   DEL    |    TRY   |
--------------------------------------------------------
|    0     |    0     |    24    |    0     |    2     |
|   100    |   596    |    24    |    33    |    3     |
|   200    |   616    |    24    |    24    |    3     |
|   300    |   600    |    24    |    20    |    3     |
|   400    |   621    |    24    |    22    |    3     |
|   500    |   620    |    24    |    32    |    3     |
|   600    |   619    |    25    |    29    |    3     |
|   700    |   643    |    25    |    24    |    3     |
|   800    |   655    |    25    |    27    |    3     |
|   900    |   651    |    25    |    26    |    3     |
|   999    |   629    |    25    |    22    |    3     |


### Analysis

Objectives :
* Extract the speed of the flow to compare it to the theoretical one
* Compare mass flow rates in output

In [11]:
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from src.data import Saver
from src.plotting import analysis
dir_path = Path('results/benchmark/')
name = 'tube-flow.h5'

store = pd.HDFStore(dir_path/name)

In [12]:
store.keys()


['/collisions_per_cell',
 '/df',
 '/pmax_per_cell',
 '/total_deleted',
 '/total_distance',
 '/total_proba']

In [26]:

df = store['df']

In [13]:
# plotting pmax evolution
pmax_per_cell = store['pmax_per_cell']

fig, ax = plt.subplots()
ax.plot(pmax_per_cell.groupby(pmax_per_cell.index).mean())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
nb_collisions_per_cell = store['collisions_per_cell']
nb_collisions_per_cell = nb_collisions_per_cell.groupby(nb_collisions_per_cell.index).sum()

In [18]:
fig, ax = plt.subplots()
plt.plot(collisions_per_cell)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fig, ax = plt.subplots()
analysis.nb_particles_evolution(ax, store['df'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
fig, ax = plt.subplots()
proba = store['total_proba']/nb_collisions_per_cell
ax.plot(proba.groupby(proba.index).sum())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
fig, ax = plt.subplots()

deleted_evo = store['total_deleted']
plt.plot(deleted_evo)

def compute_mass_flow_rate(qty, delta_time, mass):
    return qty*mass/delta_time

print('Mass flow rate : {:.3e} kg/s.'.format(compute_mass_flow_rate(mr*np.sum(deleted_evo[8000:]), delta_time = dt*2000, mass = mass)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mass flow rate : 0.000e+00 kg/s.


In [27]:
# Mach number and speed flow
min_it, max_it, step = iterations-2000, iterations, adding_period # we save every adding_period the state of the particles
iterations_list = [k for k in range(min_it, max_it, step)]
df_it = df.loc[np.isin(df.index,iterations_list) & (df['x']>0.009)]
flow_speed = np.mean(df_it['vx'])

In [28]:
print('Mean speed along the x-axis for the last cell : {:.3e} m/s'.format(flow_speed))

Mean speed along the x-axis for the last cell : 1.168e+02 m/s
